In [28]:
import pandas as pd
import numpy as np
#import config as cfg
import seaborn as sns

from sklearn.model_selection import train_test_split
import sklearn
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold
from xgboost import XGBRegressor
from xgboost import plot_importance
import datetime as dt

## 1. Product Clicks

In [53]:
df_clicks =  pd.read_csv("Tables/prod_product_viewed.csv")

In [54]:
df_clicks = df_clicks[['id', 'brand', 'category', 'event', 'event_text', 'name', 'original_timestamp', 'timestamp',\
                       'price', 'product_id', 'price']]
df_clicks

,id,brand,category,event,event_text,name,original_timestamp,timestamp,price,product_id,price
0,ajs-next-a0fa08e1d278e7cf29e4180f5942fdf7,PROARTE,Brush Sets,product_viewed,Product Viewed,PROARTE Pro Brush - (28Pcs),2022-12-28 12:56:45.816 UTC,2022-12-28 12:56:47.366 UTC,17000.0,7524675.0,17000.0
1,ajs-next-b79169f9cf90ebc151114ce3bd88be27,Wet n Wild,Sponges & Applicators,product_viewed,Product Viewed,Wet N Wild Makeup Sponge Applicator,2022-12-28 12:40:56.101 UTC,2022-12-28 12:40:56.843 UTC,509.0,7530242.0,509.0
2,ajs-next-86a9f92af2427dc3c59169b745f47aed,Ustraa,Colognes & After Shaves,product_viewed,Product Viewed,Ustraa After Dark Cologne Perfume - (100ml),2023-02-19 08:02:30.538 UTC,2023-02-19 08:02:31.199 UTC,611.0,7504965.0,611.0
3,ajs-next-838df329be126b305b37301cfbf7fcdd,The Face Shop,Sheet Masks,product_viewed,Product Viewed,The Face Shop Real Nature Green Tea Face Sheet...,2023-02-19 12:22:25.262 UTC,2023-02-19 12:22:25.271 UTC,100.0,7538505.0,100.0
4,ajs-next-5380d8638d3f205d3fde9a67931a0931,Colorbar,Eyebrow Enhancer,product_viewed,Product Viewed,Colorbar Stunning Brow Mini Eyebrow Pencil - 0...,2023-03-12 05:45:04.578 UTC,2023-03-12 05:45:04.433 UTC,239.0,7502178.0,239.0
...,...,...,...,...,...,...,...,...,...,...,...
107451,ajs-next-2eb55cb4f1abf0cae66057482d011459,AND,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,And Love Muse EDP & Pretty Vogue Mist - (2Pcs),2023-02-10 01:08:54.045 UTC,2023-02-10 01:08:54.041 UTC,1804.0,7539766.0,1804.0
107452,936F9F8D-F71C-4069-8867-956D328AF4E1,Dolce&Gabbana,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Dolce & Gabbana Pour Femme Intense EDP (50ml),2023-01-24 09:27:25.3 UTC,2023-01-24 09:27:25.884 UTC,7850.0,7545438.0,7850.0
107453,BC1CE98C-02FF-4D59-9374-72F7A2A1EF2A,Carolina Herrera,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Carolina Herrera Very Good Girl Eau De Parfum ...,2023-03-22 16:14:44.032 UTC,2023-03-22 16:14:44.271 UTC,8000.0,7506000.0,8000.0
107454,364BC995-6653-48DF-9BE1-E6768C4CD4EA,Paco Rabanne,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Paco Rabanne Pure XS for Her Eau De Parfum (50ml),2023-01-18 04:39:41.583 UTC,2023-01-18 04:39:41.833 UTC,5270.0,7506583.0,5270.0


In [55]:
df_clicks['original_timestamp'] = df_clicks['original_timestamp'].apply(lambda x: x.split(' ')[0] + ' ' + x.split(' ')[1])
df_clicks['timestamp'] = df_clicks['timestamp'].apply(lambda x: x.split(' ')[0] + ' ' + x.split(' ')[1])

df_clicks['timestamp'] = pd.to_datetime(df_clicks['timestamp'], format="%Y-%m-%d %H:%M:%S")
df_clicks['original_timestamp'] = pd.to_datetime(df_clicks['original_timestamp'], format="%Y-%m-%d %H:%M:%S")
df_clicks

,id,brand,category,event,event_text,name,original_timestamp,timestamp,price,product_id,price
0,ajs-next-a0fa08e1d278e7cf29e4180f5942fdf7,PROARTE,Brush Sets,product_viewed,Product Viewed,PROARTE Pro Brush - (28Pcs),2022-12-28 12:56:45.816,2022-12-28 12:56:47.366,17000.0,7524675.0,17000.0
1,ajs-next-b79169f9cf90ebc151114ce3bd88be27,Wet n Wild,Sponges & Applicators,product_viewed,Product Viewed,Wet N Wild Makeup Sponge Applicator,2022-12-28 12:40:56.101,2022-12-28 12:40:56.843,509.0,7530242.0,509.0
2,ajs-next-86a9f92af2427dc3c59169b745f47aed,Ustraa,Colognes & After Shaves,product_viewed,Product Viewed,Ustraa After Dark Cologne Perfume - (100ml),2023-02-19 08:02:30.538,2023-02-19 08:02:31.199,611.0,7504965.0,611.0
3,ajs-next-838df329be126b305b37301cfbf7fcdd,The Face Shop,Sheet Masks,product_viewed,Product Viewed,The Face Shop Real Nature Green Tea Face Sheet...,2023-02-19 12:22:25.262,2023-02-19 12:22:25.271,100.0,7538505.0,100.0
4,ajs-next-5380d8638d3f205d3fde9a67931a0931,Colorbar,Eyebrow Enhancer,product_viewed,Product Viewed,Colorbar Stunning Brow Mini Eyebrow Pencil - 0...,2023-03-12 05:45:04.578,2023-03-12 05:45:04.433,239.0,7502178.0,239.0
...,...,...,...,...,...,...,...,...,...,...,...
107451,ajs-next-2eb55cb4f1abf0cae66057482d011459,AND,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,And Love Muse EDP & Pretty Vogue Mist - (2Pcs),2023-02-10 01:08:54.045,2023-02-10 01:08:54.041,1804.0,7539766.0,1804.0
107452,936F9F8D-F71C-4069-8867-956D328AF4E1,Dolce&Gabbana,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Dolce & Gabbana Pour Femme Intense EDP (50ml),2023-01-24 09:27:25.300,2023-01-24 09:27:25.884,7850.0,7545438.0,7850.0
107453,BC1CE98C-02FF-4D59-9374-72F7A2A1EF2A,Carolina Herrera,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Carolina Herrera Very Good Girl Eau De Parfum ...,2023-03-22 16:14:44.032,2023-03-22 16:14:44.271,8000.0,7506000.0,8000.0
107454,364BC995-6653-48DF-9BE1-E6768C4CD4EA,Paco Rabanne,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Paco Rabanne Pure XS for Her Eau De Parfum (50ml),2023-01-18 04:39:41.583,2023-01-18 04:39:41.833,5270.0,7506583.0,5270.0


In [56]:
# calculate the date 14 days before today's date
today = dt.date.today()
last_14_days = today - dt.timedelta(days=14)
print(str(last_14_days))

df_clicks['date'] = df_clicks['original_timestamp'].apply(lambda x: x.date())
df_clicks_fortnight = df_clicks[df_clicks['date'] > last_14_days]
df_clicks_fortnight

2023-03-17


,id,brand,category,event,event_text,name,original_timestamp,timestamp,price,product_id,price,date
58,ajs-next-de0166e27c60c0cc472582b34819e299,Bioderma,Makeup Remover,product_viewed,Product Viewed,Bioderma Sensibio H2O Daily Soothing Cleanser ...,2023-03-18 10:17:49.353,2023-03-18 10:17:47.946,355.0,7545921.0,355.0,2023-03-18
71,ajs-next-6c186d678c0cd43dce39703d3097326e,Mirah Belle,Hand Sanitizers,product_viewed,Product Viewed,Mirah Belle Surface Disinfectant Spray (200ml),2023-03-19 22:32:50.304,2023-03-19 22:32:50.332,150.0,7507155.0,150.0,2023-03-19
72,ajs-next-d22c41c040eccd6fddac69b803b44c51,Organic Harvest,Hand Creams,product_viewed,Product Viewed,Organic Harvest Cranberry Hand Cream (50g),2023-03-19 17:17:23.383,2023-03-19 17:17:23.601,196.0,7517185.0,196.0,2023-03-19
80,ajs-next-435de2d8ee09ffd4cf75a26cca907594,PAC,Eye Brush,product_viewed,Product Viewed,PAC Eyeshadow Application Brush - 382 (1Pc),2023-03-22 11:15:44.296,2023-03-22 11:15:44.783,435.0,7541811.0,435.0,2023-03-22
84,ajs-next-4689d939a1ac1798d76e8894b4ded728,Layer'r,Body Mists & Sprays,product_viewed,Product Viewed,Layer'r Wottagirl! Body Splash Set (4Pcs),2023-03-21 10:17:49.408,2023-03-21 10:17:50.059,320.0,7549401.0,320.0,2023-03-21
...,...,...,...,...,...,...,...,...,...,...,...,...
107276,3822D69A-6505-41C1-8421-BD8397EA125E,Versace,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Versace Dylan Turquoise Eau De Toilette (50ml),2023-03-22 17:16:36.452,2023-03-22 17:16:36.603,6150.0,7515961.0,6150.0,2023-03-22
107301,8524FC47-CB8B-4772-B67D-69031987A1B7,Plum,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Plum Bodylovin Orchid You Not Eau De Parfum - ...,2023-03-22 18:07:05.609,2023-03-22 18:07:06.367,950.0,7528077.0,950.0,2023-03-22
107337,ajs-next-d14cc8169b76857465d46e7490561418,Michael Kors,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Michael Kors Sexy Amber Eau de Parfum (50ml),2023-03-20 11:44:12.111,2023-03-20 11:44:12.540,3810.0,7506349.0,3810.0,2023-03-20
107341,b9bddf78-a058-435f-aee0-59bfdf8f92bc,Estee Lauder,Womens Perfumes (EDT & EDP),product_viewed,Product Viewed,Estee Lauder Modern Muse Eau De Parfum (100ml),2023-03-22 13:44:24.978,2023-03-22 13:44:25.272,8075.0,7533342.0,8075.0,2023-03-22


In [57]:
df_clicks_fortnight.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8296 entries, 58 to 107453
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  8296 non-null   object        
 1   brand               8197 non-null   object        
 2   category            8197 non-null   object        
 3   event               8296 non-null   object        
 4   event_text          8296 non-null   object        
 5   name                8197 non-null   object        
 6   original_timestamp  8296 non-null   datetime64[ns]
 7   timestamp           8296 non-null   datetime64[ns]
 8   price               8206 non-null   float64       
 9   product_id          8197 non-null   float64       
 10  price               8206 non-null   float64       
 11  date                8296 non-null   object        
dtypes: datetime64[ns](2), float64(3), object(7)
memory usage: 842.6+ KB


## Brand-wise Clicks

In [58]:
df_clicks_brand = df_clicks_fortnight.copy()
df_clicks_brand['brand'].value_counts()

Lakme                  458
Maybelline New York    458
M.A.C                  414
Estee Lauder           300
L'Oreal Paris          295
                      ... 
Coach                    1
Jean Paul Gaultier       1
Urban Veda               1
BrownSkin Beauty         1
Maxxcode                 1
Name: brand, Length: 391, dtype: int64

In [59]:
df_clicks_brand['clicks'] = np.zeros(len(df_clicks_brand))
df_clicks_brand = df_clicks_brand.groupby('brand', as_index=False).agg({'clicks': 'count'})
df_clicks_brand.sort_values('clicks', ascending=False)

,brand,clicks
195,Lakme,458
224,Maybelline New York,458
209,M.A.C,414
114,Estee Lauder,300
186,L'Oreal Paris,295
...,...,...
223,Maxxcode,1
215,Majestique,1
210,M.Micallef,1
203,Lottie London,1


## Category-wise Clicks

In [62]:
brand_to_category_dict = dict(zip(df_clicks_fortnight['brand'], df_clicks_fortnight['category']))
brand_to_category_dict

{'Bioderma': 'Face Moisturiser',
 'Mirah Belle': 'Body Wash & Shower Gel',
 'Organic Harvest': 'Serums & Essences',
 'PAC': 'Face Moisturiser',
 "Layer'r": 'Body Mists & Sprays',
 'L’Oreal Paris': 'Liquid Lipstick',
 'CLINIQUE': 'Serums & Essences',
 'Indica': 'Hair Colour',
 'Vaseline': 'Lotions & Cream',
 'MYGLAMM': 'Liquid Lipstick',
 'Star Struck by Sunny Leone': 'Lipstick',
 'mCaffeine': 'Body Wash & Shower Gel',
 'Vega': 'Trimmers',
 'Colorbar': 'Face Moisturiser',
 'Skin Elements': 'Intimate Care',
 'Nivea': 'Mens Deodorants & Roll-Ons',
 'Babila': 'Face Brush',
 'Bold Care': 'Shampoo',
 'ASA beauty': 'Lip Stain',
 'Farmacy Beauty': 'Serums & Essences',
 'Plum': 'Womens Perfumes (EDT & EDP)',
 'M.A.C': 'Face Moisturizer',
 'Mamaearth': 'Mens Perfumes (EDT & EDP)',
 'Soulflower': 'Serums & Essences',
 'Gillette': 'Razors & Cartridges',
 'India Grooming Club': 'Shavers',
 'Anastasia Beverly Hills': 'Eye Palettes',
 "Re'equil": 'Serums & Essences',
 'Beardo': 'Mens Deodorants & Rol

In [63]:
len(brand_to_category_dict.keys())

392

In [64]:
df_clicks_category = df_clicks_fortnight.copy()
df_clicks_category['clicks'] = np.zeros(len(df_clicks_category))
df_clicks_category = df_clicks_category.groupby('category', as_index=False).agg({'clicks': 'count'})
df_clicks_category.sort_values('clicks', ascending=False)

,category,clicks
113,Lipstick,725
114,Liquid Lipstick,512
38,Concealer,444
73,Foundation,427
147,Serums & Essences,414
...,...,...
9,Beard Balm,1
154,Shaving Foams,1
18,Beard Wax,1
46,Eye Care Eye Masks,1


## 2. Order Completed